In [ ]:
!pip install mumin[all]==1.6.2 torchmetrics==0.7.2 --quiet
!pip install dgl-cu111==0.7.2 -f https://data.dgl.ai/wheels/repo.html --quiet
!pip install pickle5

     |████████████████████████████████| 397 kB 8.2 MB/s 
     |████████████████████████████████| 211 kB 50.2 MB/s 
     |████████████████████████████████| 62 kB 678 kB/s 
     |████████████████████████████████| 6.2 MB 7.1 MB/s 
     |████████████████████████████████| 4.7 MB 54.0 MB/s 
     |████████████████████████████████| 281 kB 59.2 MB/s 
     |████████████████████████████████| 93 kB 1.6 MB/s 
     |████████████████████████████████| 7.4 MB 7.4 MB/s 
     |████████████████████████████████| 81 kB 8.3 MB/s 
     |████████████████████████████████| 6.6 MB 33.0 MB/s 
     |████████████████████████████████| 120 kB 63.5 MB/s 
     |████████████████████████████████| 165.0 MB 28 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 256 kB 5.2 MB/s 


In [ ]:
from mumin import MuminDataset
from google.colab import drive
from getpass import getpass
from pathlib import Path
import shutil
import re
import pickle5 as pickle
import pandas as pd
import numpy as np

In [ ]:
drive_dir = Path('drive')
drive.mount(str(drive_dir.resolve()), force_remount=True)
drive_content_dir = [child for child in drive_dir.iterdir() 
                     if re.search(r'My ?Drive', str(child.stem)) is not None][0]
shutil.copy(drive_content_dir / 'mumin-small.zip', 'mumin-small.zip')

Mounted at /content/drive


'mumin-small.zip'

In [ ]:
with open ('/content/drive/MyDrive/mumin-small/emb/multilingual_emb_sml_tweet.pkl', 'rb') as f:
    file1 = pd.DataFrame(pickle.load(f))
with open ('/content/drive/MyDrive/mumin-small/emb/text_emb_sml_reply.pkl', 'rb') as f:
    file2 = pd.DataFrame(pickle.load(f))
with open ('/content/drive/MyDrive/mumin-small/emb/multilingual_emb_sml_reply.pkl', 'rb') as f:
    file3 = pd.DataFrame(pickle.load(f))
with open ('/content/drive/MyDrive/mumin-small/emb/text_emb_sml_tweet.pkl', 'rb') as f:
    file4 = pd.DataFrame(pickle.load(f))


In [ ]:
drive_dir = Path('drive')
drive.mount(str(drive_dir.resolve()), force_remount=True)
drive_content_dir = [child for child in drive_dir.iterdir() 
                     if re.search(r'My ?Drive', str(child.stem)) is not None][0]
shutil.copy(drive_content_dir / 'mumin-small.zip', 'mumin-small.zip')

Mounted at /content/drive


'mumin-small.zip'

In [ ]:
dataset = MuminDataset('mumin-small.zip')
dataset

MuminDataset(size=small, compiled=False)

In [ ]:
dataset.compile()

INFO:mumin.dataset:Loading dataset


MuminDataset(num_nodes=392,419, num_relations=483,029, size='small', compiled=True)

In [ ]:
with open("/content/drive/MyDrive/mumin-small/emb/tweet_emb.pickle", "wb") as fp:   #Pickling
    pickle.dump(dataset.nodes['tweet'], fp)  
with open("/content/drive/MyDrive/mumin-small/emb/reply_emb.pickle", "wb") as fp:   #Pickling
    pickle.dump(dataset.nodes['reply'], fp)  


In [ ]:
drive_dir = Path('drive')
drive.mount(str(drive_dir.resolve()), force_remount=True)

Mounted at /content/drive


In [ ]:
drive_content_dir = [child for child in drive_dir.iterdir() 
                     if re.search(r'My ?Drive', str(child.stem)) is not None][0]
shutil.copy('mumin-small.zip', drive_content_dir / 'mumin-small_reduced.zip')

PosixPath('drive/MyDrive/mumin-small_reduced.zip')

In [ ]:
drive_dir = Path('drive')
drive_content_dir = [child for child in drive_dir.iterdir() 
                     if re.search(r'My ?Drive', str(child.stem)) is not None][0]
drive.mount(str(drive_dir.resolve()), force_remount=True)
shutil.copy('mumin-small.zip', drive_content_dir / 'mumin-small_reduced.zip')

Mounted at /content/drive


PosixPath('drive/MyDrive/mumin-small_reduced.zip')

In [ ]:
drive_dir = Path('drive')
drive.mount(str(drive_dir.resolve()), force_remount=True)
drive_content_dir = [child for child in drive_dir.iterdir() 
                     if re.search(r'My ?Drive', str(child.stem)) is not None][0]
shutil.copy(drive_content_dir / 'mumin-small_reduced.zip', 'mumin-small_reduced.zip')

Mounted at /content/drive


'mumin-small_reduced.zip'

In [ ]:
dataset = MuminDataset('mumin-small_reduced.zip')
dataset

MuminDataset(size=small, compiled=False)

In [ ]:
dataset.compile()

INFO:mumin.dataset:Loading dataset


MuminDataset(num_nodes=392,419, num_relations=483,029, size='small', compiled=True)

In [ ]:
len(dataset.nodes['tweet']['mbert_emb'][0])

KeyError: ignored

In [ ]:
from PIL import Image
import itertools as it

<center><img src="https://filedn.com/lRBwPhPxgV74tO0rDoe8SpH/metagraph.png" alt="meta graph of the MuMiN dataset" width="60%"/></center>

In [ ]:
list(dataset.nodes.keys())

NameError: ignored

## 5. Building a graph misinformation classifier

In [ ]:
from mumin import save_dgl_graph, load_dgl_graph
import dgl
import dgl.nn.pytorch as dglnn
import dgl.dataloading as D
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch
import torchmetrics as tm
import networkx as nx
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from collections import defaultdict

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


Using backend: pytorch


In [ ]:
if 'dgl_graph' not in globals():
    dgl_graph = dataset.to_dgl()
dgl_graph

INFO:mumin.dataset:Outputting to DGL


Graph(num_nodes={'article': 1497, 'claim': 2127, 'hashtag': 28820, 'image': 1036, 'reply': 186602, 'tweet': 4178, 'user': 158995},
      num_edges={('article', 'has_article_inv', 'tweet'): 1969, ('claim', 'discusses_inv', 'tweet'): 4890, ('hashtag', 'has_hashtag_inv', 'tweet'): 2343, ('hashtag', 'has_hashtag_inv', 'user'): 52420, ('image', 'has_image_inv', 'tweet'): 1045, ('reply', 'posted_inv', 'user'): 186602, ('reply', 'quote_of', 'tweet'): 93926, ('reply', 'reply_to', 'tweet'): 84271, ('tweet', 'discusses', 'claim'): 4890, ('tweet', 'has_article', 'article'): 1969, ('tweet', 'has_hashtag', 'hashtag'): 2343, ('tweet', 'has_image', 'image'): 1045, ('tweet', 'mentions', 'user'): 1134, ('tweet', 'posted_inv', 'user'): 4178, ('tweet', 'quote_of_inv', 'reply'): 93926, ('tweet', 'reply_to_inv', 'reply'): 84271, ('tweet', 'retweeted_inv', 'user'): 13710, ('user', 'follows', 'user'): 19886, ('user', 'follows_inv', 'user'): 19886, ('user', 'has_hashtag', 'hashtag'): 52420, ('user', 'mentions

In [ ]:
print('Node types in the DGL graph:')
print(dgl_graph.ntypes)
print('\nRelation types in the DGL graph:')
dgl_graph.canonical_etypes

Node types in the DGL graph:
['article', 'claim', 'hashtag', 'image', 'reply', 'tweet', 'user']

Relation types in the DGL graph:


[('article', 'has_article_inv', 'tweet'),
 ('claim', 'discusses_inv', 'tweet'),
 ('hashtag', 'has_hashtag_inv', 'tweet'),
 ('hashtag', 'has_hashtag_inv', 'user'),
 ('image', 'has_image_inv', 'tweet'),
 ('reply', 'posted_inv', 'user'),
 ('reply', 'quote_of', 'tweet'),
 ('reply', 'reply_to', 'tweet'),
 ('tweet', 'discusses', 'claim'),
 ('tweet', 'has_article', 'article'),
 ('tweet', 'has_hashtag', 'hashtag'),
 ('tweet', 'has_image', 'image'),
 ('tweet', 'mentions', 'user'),
 ('tweet', 'posted_inv', 'user'),
 ('tweet', 'quote_of_inv', 'reply'),
 ('tweet', 'reply_to_inv', 'reply'),
 ('tweet', 'retweeted_inv', 'user'),
 ('user', 'follows', 'user'),
 ('user', 'follows_inv', 'user'),
 ('user', 'has_hashtag', 'hashtag'),
 ('user', 'mentions', 'user'),
 ('user', 'mentions_inv', 'tweet'),
 ('user', 'mentions_inv', 'user'),
 ('user', 'posted', 'reply'),
 ('user', 'posted', 'tweet'),
 ('user', 'retweeted', 'tweet')]

In [ ]:
plt.figure(figsize=(10, 7))
metagraph = dgl_graph.metagraph()
nx.draw_networkx(metagraph, 
                 pos=nx.shell_layout(metagraph), 
                 node_color='white', 
                 node_size=3000,
                 arrows=False)

## 'user', 'posted', 'tweet'

In [ ]:
rel1 = ('article', 'has_article_inv', 'tweet')
rel2 = ('user', 'retweeted', 'tweet')
subgraph = dgl.edge_type_subgraph(dgl_graph, etypes=[rel1, rel2]).to('cuda')
subgraph

train_mask = subgraph.nodes['tweet'].data['train_mask']
val_mask = subgraph.nodes['tweet'].data['val_mask']
test_mask = subgraph.nodes['tweet'].data['test_mask']

class SAGEClassifier(nn.Module):
    def __init__(self, hidden_dim: int = 500):
        super().__init__()
        feats1 = subgraph.nodes['article'].data['feat'].shape[-1]
        feats2 = subgraph.nodes['user'].data['feat'].shape[-1]
        feats3 = subgraph.nodes['tweet'].data['feat'].shape[-1]
        self.conv = dglnn.HeteroGraphConv({
                'has_article_inv': dglnn.SAGEConv(in_feats=(feats1,feats3), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU()),
                'retweeted': dglnn.SAGEConv(in_feats=(feats2,feats3), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU())
                }, aggregate='sum')

        self.clf = nn.Sequential(
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )


    def forward(self, graph, inputs):
        x = self.conv(graph, inputs)
        x = self.clf(x['tweet'])
        return x

gnn = SAGEClassifier().cuda()
gnn

def forward_pass() -> dict:
    '''A forward pass of the graph neural network.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Set the GNN to training mode
    gnn.train()

    # Get the input features and the output labels
    input_feats = dict(
        article=subgraph.nodes['article'].data['feat'].float().cuda(),
        user=subgraph.nodes['user'].data['feat'].float().cuda(),
        tweet=subgraph.nodes['tweet'].data['feat'].float().cuda()
    )
    output_labels = subgraph.nodes['tweet'].data['label'].cuda()

    # Forward propagation
    #logits = gnn(subgraph, input_feats)['tweet']
    logits = gnn(subgraph, input_feats).squeeze()

    # Compute loss
    loss = F.binary_cross_entropy_with_logits(
        input=logits[train_mask],
        target=output_labels.float()[train_mask]
    )

    # Compute training metrics
    scores = scorer(logits[train_mask].ge(0), output_labels[train_mask])
    misinformation_f1 = scores[0]
    factual_f1 = scores[1]

    return dict(loss=loss, 
                misinformation_f1=misinformation_f1, 
                factual_f1=factual_f1)

def evaluate(split: str) -> dict:
    '''Evaluate the graph neural network.

    Args:
        split (str):
            The split to evaluate the GNN on. Can be 'val' or 'test'.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Get the correct mask, depending on the value of `split`
    mask = val_mask if split == 'val' else test_mask

    gnn.eval()
    with torch.no_grad():

        # Get the input features and the output labels
        input_feats = dict(
            article=subgraph.nodes['article'].data['feat'].float().cuda(),
            user=subgraph.nodes['user'].data['feat'].float().cuda(),
            tweet=subgraph.nodes['tweet'].data['feat'].float().cuda()
        )
        
        output_labels = subgraph.nodes['tweet'].data['label'].cuda()

        # Forward propagation
        logits = gnn(subgraph, input_feats).squeeze()

        # Compute validation loss
        val_loss = F.binary_cross_entropy_with_logits(
            input=logits[mask],
            target=output_labels.float()[mask]
        ).cpu().item()

        # Compute validation metrics
        scores = scorer(logits[mask].ge(0), output_labels[mask])
        val_misinformation_f1 = scores[0].cpu().item()
        val_factual_f1 = scores[1].cpu().item()

    return dict(loss=val_loss, 
                misinformation_f1=val_misinformation_f1, 
                factual_f1=val_factual_f1)

# Initialise optimiser
opt = optim.AdamW(gnn.parameters(), lr=3e-4)

# Initialise scorer
scorer = tm.classification.f_beta.F1Score(num_classes=2, average='none').cuda()

# Initialise dictionary containing validation scores
val_scores = defaultdict(list)

# Initialise progress bar
epoch_pbar = tqdm(range(1000), desc='Training')

for epoch in epoch_pbar:

    # Reset the gradients
    opt.zero_grad()

    # Forward propagation
    train_results = forward_pass()

    # Backward propagation
    train_results['loss'].backward()

    # Update gradients
    opt.step()

    # Evaluate the model
    val_results = evaluate('val')

    # Store the validation scores
    for metric in ['loss', 'misinformation_f1', 'factual_f1']:
        val_scores[metric].append(val_results[metric])

    # Update progress bar description
    if epoch % 25 == 0 and epoch > 0:
        val_loss = np.mean(val_scores['loss'])
        val_misinformation_f1 = np.mean(val_scores['misinformation_f1'])
        val_factual_f1 = np.mean(val_scores['factual_f1'])
        desc = (f'Training - '
                f'loss {train_results["loss"]:.3f} - '
                f'factual_f1 {train_results["factual_f1"]:.3f} - '
                f'misinfo_f1 {train_results["misinformation_f1"]:.3f} - '
                f'val_loss {val_loss:.3f} - '
                f'val_factual_f1 {val_factual_f1:.3f} - '
                f'val_misinfo_f1 {val_misinformation_f1:.3f}')
        epoch_pbar.set_description(desc)
        val_scores = defaultdict(list)

test_results = evaluate('test')
macro_f1 = np.mean([test_results['factual_f1'],
                    test_results['misinformation_f1']])
print('*** Test results ***')
print(f'Factual F1: {test_results["factual_f1"]:.4f}')
print(f'Misinformation F1: {test_results["misinformation_f1"]:.4f}')
print(f'Macro-average F1: {macro_f1:.4f}')

Training:   0%|          | 0/1000 [00:00<?, ?it/s]

*** Test results ***
Factual F1: 0.1842
Misinformation F1: 0.9265
Macro-average F1: 0.5554


In [ ]:
rel1 = ('article', 'has_article_inv', 'tweet')
rel2 = ('user', 'retweeted', 'tweet')
rel3 = ('user', 'posted', 'tweet')

subgraph = dgl.edge_type_subgraph(dgl_graph, etypes=[rel1, rel2, rel3]).to('cuda')
subgraph

train_mask = subgraph.nodes['tweet'].data['train_mask']
val_mask = subgraph.nodes['tweet'].data['val_mask']
test_mask = subgraph.nodes['tweet'].data['test_mask']

class SAGEClassifier(nn.Module):
    def __init__(self, hidden_dim: int = 500):
        super().__init__()
        feats1 = subgraph.nodes['article'].data['feat'].shape[-1]
        feats2 = subgraph.nodes['user'].data['feat'].shape[-1]
        feats3 = subgraph.nodes['tweet'].data['feat'].shape[-1]
        self.conv = dglnn.HeteroGraphConv({
                'has_article_inv': dglnn.SAGEConv(in_feats=(feats1,feats3), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU()),
                'retweeted': dglnn.SAGEConv(in_feats=(feats2,feats3), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU()),
                'posted': dglnn.SAGEConv(in_feats=(feats2,feats3), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU())
                }, aggregate='sum')

        self.clf = nn.Sequential(
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )


    def forward(self, graph, inputs):
        x = self.conv(graph, inputs)
        x = self.clf(x['tweet'])
        return x

gnn = SAGEClassifier().cuda()
gnn

def forward_pass() -> dict:
    '''A forward pass of the graph neural network.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Set the GNN to training mode
    gnn.train()

    # Get the input features and the output labels
    input_feats = dict(
        article=subgraph.nodes['article'].data['feat'].float().cuda(),
        user=subgraph.nodes['user'].data['feat'].float().cuda(),
        tweet=subgraph.nodes['tweet'].data['feat'].float().cuda()
    )
    output_labels = subgraph.nodes['tweet'].data['label'].cuda()

    # Forward propagation
    logits = gnn(subgraph, input_feats).squeeze()

    # Compute loss
    loss = F.binary_cross_entropy_with_logits(
        input=logits[train_mask],
        target=output_labels.float()[train_mask]
    )

    # Compute training metrics
    scores = scorer(logits[train_mask].ge(0), output_labels[train_mask])
    misinformation_f1 = scores[0]
    factual_f1 = scores[1]

    return dict(loss=loss, 
                misinformation_f1=misinformation_f1, 
                factual_f1=factual_f1)

def evaluate(split: str) -> dict:
    '''Evaluate the graph neural network.

    Args:
        split (str):
            The split to evaluate the GNN on. Can be 'val' or 'test'.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Get the correct mask, depending on the value of `split`
    mask = val_mask if split == 'val' else test_mask

    gnn.eval()
    with torch.no_grad():

        # Get the input features and the output labels
        input_feats = dict(
            article=subgraph.nodes['article'].data['feat'].float().cuda(),
            user=subgraph.nodes['user'].data['feat'].float().cuda(),
            tweet=subgraph.nodes['tweet'].data['feat'].float().cuda()
        )
        
        output_labels = subgraph.nodes['tweet'].data['label'].cuda()

        # Forward propagation
        logits = gnn(subgraph, input_feats).squeeze()

        # Compute validation loss
        val_loss = F.binary_cross_entropy_with_logits(
            input=logits[mask],
            target=output_labels.float()[mask]
        ).cpu().item()

        # Compute validation metrics
        scores = scorer(logits[mask].ge(0), output_labels[mask])
        val_misinformation_f1 = scores[0].cpu().item()
        val_factual_f1 = scores[1].cpu().item()

    return dict(loss=val_loss, 
                misinformation_f1=val_misinformation_f1, 
                factual_f1=val_factual_f1)

# Initialise optimiser
opt = optim.AdamW(gnn.parameters(), lr=3e-4)

# Initialise scorer
scorer = tm.classification.f_beta.F1Score(num_classes=2, average='none').cuda()

# Initialise dictionary containing validation scores
val_scores = defaultdict(list)

# Initialise progress bar
epoch_pbar = tqdm(range(1000), desc='Training')

for epoch in epoch_pbar:

    # Reset the gradients
    opt.zero_grad()

    # Forward propagation
    train_results = forward_pass()

    # Backward propagation
    train_results['loss'].backward()

    # Update gradients
    opt.step()

    # Evaluate the model
    val_results = evaluate('val')

    # Store the validation scores
    for metric in ['loss', 'misinformation_f1', 'factual_f1']:
        val_scores[metric].append(val_results[metric])

    # Update progress bar description
    if epoch % 25 == 0 and epoch > 0:
        val_loss = np.mean(val_scores['loss'])
        val_misinformation_f1 = np.mean(val_scores['misinformation_f1'])
        val_factual_f1 = np.mean(val_scores['factual_f1'])
        desc = (f'Training - '
                f'loss {train_results["loss"]:.3f} - '
                f'factual_f1 {train_results["factual_f1"]:.3f} - '
                f'misinfo_f1 {train_results["misinformation_f1"]:.3f} - '
                f'val_loss {val_loss:.3f} - '
                f'val_factual_f1 {val_factual_f1:.3f} - '
                f'val_misinfo_f1 {val_misinformation_f1:.3f}')
        epoch_pbar.set_description(desc)
        val_scores = defaultdict(list)

test_results = evaluate('test')
macro_f1 = np.mean([test_results['factual_f1'],
                    test_results['misinformation_f1']])
print('*** Test results ***')
print(f'Factual F1: {test_results["factual_f1"]:.4f}')
print(f'Misinformation F1: {test_results["misinformation_f1"]:.4f}')
print(f'Macro-average F1: {macro_f1:.4f}')

Training:   0%|          | 0/1000 [00:00<?, ?it/s]

*** Test results ***
Factual F1: 0.2121
Misinformation F1: 0.9391
Macro-average F1: 0.5756


In [ ]:
rel1 = ('article', 'has_article_inv', 'tweet')
rel2 = ('user', 'retweeted', 'tweet')
rel3 = ('user', 'posted', 'tweet')
rel4 = ('hashtag', 'has_hashtag_inv', 'tweet')

subgraph = dgl.edge_type_subgraph(dgl_graph, etypes=[rel1, rel2, rel3, rel4]).to('cuda')
subgraph

train_mask = subgraph.nodes['tweet'].data['train_mask']
val_mask = subgraph.nodes['tweet'].data['val_mask']
test_mask = subgraph.nodes['tweet'].data['test_mask']

class SAGEClassifier(nn.Module):
    def __init__(self, hidden_dim: int = 500):
        super().__init__()
        feats1 = subgraph.nodes['article'].data['feat'].shape[-1]
        feats2 = subgraph.nodes['user'].data['feat'].shape[-1]
        feats3 = subgraph.nodes['hashtag'].data['feat'].shape[-1]
        feats4 = subgraph.nodes['tweet'].data['feat'].shape[-1]
        self.conv = dglnn.HeteroGraphConv({
                'has_article_inv': dglnn.SAGEConv(in_feats=(feats1,feats4), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU()),
                'retweeted': dglnn.SAGEConv(in_feats=(feats2,feats4), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU()),
                'posted': dglnn.SAGEConv(in_feats=(feats2,feats4), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU()),
                'has_hashtag_inv': dglnn.SAGEConv(in_feats=(feats3,feats4), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU())
                }, aggregate='sum')

        self.clf = nn.Sequential(
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )


    def forward(self, graph, inputs):
        x = self.conv(graph, inputs)
        x = self.clf(x['tweet'])
        return x

gnn = SAGEClassifier().cuda()
gnn

def forward_pass() -> dict:
    '''A forward pass of the graph neural network.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Set the GNN to training mode
    gnn.train()

    # Get the input features and the output labels
    input_feats = dict(
        article=subgraph.nodes['article'].data['feat'].float().cuda(),
        user=subgraph.nodes['user'].data['feat'].float().cuda(),
        tweet=subgraph.nodes['tweet'].data['feat'].float().cuda(),
        hashtag=subgraph.nodes['hashtag'].data['feat'].float().cuda()
        )
    output_labels = subgraph.nodes['tweet'].data['label'].cuda()

    # Forward propagation
    logits = gnn(subgraph, input_feats).squeeze()

    # Compute loss
    loss = F.binary_cross_entropy_with_logits(
        input=logits[train_mask],
        target=output_labels.float()[train_mask]
    )

    # Compute training metrics
    scores = scorer(logits[train_mask].ge(0), output_labels[train_mask])
    misinformation_f1 = scores[0]
    factual_f1 = scores[1]

    return dict(loss=loss, 
                misinformation_f1=misinformation_f1, 
                factual_f1=factual_f1)

def evaluate(split: str) -> dict:
    '''Evaluate the graph neural network.

    Args:
        split (str):
            The split to evaluate the GNN on. Can be 'val' or 'test'.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Get the correct mask, depending on the value of `split`
    mask = val_mask if split == 'val' else test_mask

    gnn.eval()
    with torch.no_grad():

        # Get the input features and the output labels
        input_feats = dict(
            article=subgraph.nodes['article'].data['feat'].float().cuda(),
            user=subgraph.nodes['user'].data['feat'].float().cuda(),
            tweet=subgraph.nodes['tweet'].data['feat'].float().cuda(),
            hashtag=subgraph.nodes['hashtag'].data['feat'].float().cuda()
        )
        
        output_labels = subgraph.nodes['tweet'].data['label'].cuda()

        # Forward propagation
        logits = gnn(subgraph, input_feats).squeeze()

        # Compute validation loss
        val_loss = F.binary_cross_entropy_with_logits(
            input=logits[mask],
            target=output_labels.float()[mask]
        ).cpu().item()

        # Compute validation metrics
        scores = scorer(logits[mask].ge(0), output_labels[mask])
        val_misinformation_f1 = scores[0].cpu().item()
        val_factual_f1 = scores[1].cpu().item()

    return dict(loss=val_loss, 
                misinformation_f1=val_misinformation_f1, 
                factual_f1=val_factual_f1)

# Initialise optimiser
opt = optim.AdamW(gnn.parameters(), lr=3e-4)

# Initialise scorer
scorer = tm.classification.f_beta.F1Score(num_classes=2, average='none').cuda()

# Initialise dictionary containing validation scores
val_scores = defaultdict(list)

# Initialise progress bar
epoch_pbar = tqdm(range(1000), desc='Training')

for epoch in epoch_pbar:

    # Reset the gradients
    opt.zero_grad()

    # Forward propagation
    train_results = forward_pass()

    # Backward propagation
    train_results['loss'].backward()

    # Update gradients
    opt.step()

    # Evaluate the model
    val_results = evaluate('val')

    # Store the validation scores
    for metric in ['loss', 'misinformation_f1', 'factual_f1']:
        val_scores[metric].append(val_results[metric])

    # Update progress bar description
    if epoch % 25 == 0 and epoch > 0:
        val_loss = np.mean(val_scores['loss'])
        val_misinformation_f1 = np.mean(val_scores['misinformation_f1'])
        val_factual_f1 = np.mean(val_scores['factual_f1'])
        desc = (f'Training - '
                f'loss {train_results["loss"]:.3f} - '
                f'factual_f1 {train_results["factual_f1"]:.3f} - '
                f'misinfo_f1 {train_results["misinformation_f1"]:.3f} - '
                f'val_loss {val_loss:.3f} - '
                f'val_factual_f1 {val_factual_f1:.3f} - '
                f'val_misinfo_f1 {val_misinformation_f1:.3f}')
        epoch_pbar.set_description(desc)
        val_scores = defaultdict(list)

test_results = evaluate('test')
macro_f1 = np.mean([test_results['factual_f1'],
                    test_results['misinformation_f1']])
print('*** Test results ***')
print(f'Factual F1: {test_results["factual_f1"]:.4f}')
print(f'Misinformation F1: {test_results["misinformation_f1"]:.4f}')
print(f'Macro-average F1: {macro_f1:.4f}')

Training:   0%|          | 0/1000 [00:00<?, ?it/s]

*** Test results ***
Factual F1: 0.2222
Misinformation F1: 0.9249
Macro-average F1: 0.5736


In [ ]:
rel1 = ('article', 'has_article_inv', 'tweet')
rel2 = ('user', 'retweeted', 'tweet')
rel3 = ('user', 'posted', 'tweet')
rel4 = ('hashtag', 'has_hashtag_inv', 'tweet')
rel5 = ('image', 'has_image_inv', 'tweet')

subgraph = dgl.edge_type_subgraph(dgl_graph, etypes=[rel1, rel2, rel3, rel4, rel5]).to('cuda')
subgraph

train_mask = subgraph.nodes['tweet'].data['train_mask']
val_mask = subgraph.nodes['tweet'].data['val_mask']
test_mask = subgraph.nodes['tweet'].data['test_mask']

class SAGEClassifier(nn.Module):
    def __init__(self, hidden_dim: int = 500):
        super().__init__()
        feats1 = subgraph.nodes['article'].data['feat'].shape[-1]
        feats2 = subgraph.nodes['user'].data['feat'].shape[-1]
        feats3 = subgraph.nodes['hashtag'].data['feat'].shape[-1]
        feats4 = subgraph.nodes['image'].data['feat'].shape[-1]
        feats5 = subgraph.nodes['tweet'].data['feat'].shape[-1]
        self.conv = dglnn.HeteroGraphConv({
                'has_article_inv': dglnn.SAGEConv(in_feats=(feats1,feats5), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU()),
                'retweeted': dglnn.SAGEConv(in_feats=(feats2,feats5), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU()),
                'posted': dglnn.SAGEConv(in_feats=(feats2,feats5), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU()),
                'has_hashtag_inv': dglnn.SAGEConv(in_feats=(feats3,feats5), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU()),
                'has_image_inv': dglnn.SAGEConv(in_feats=(feats4,feats5), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU())
                }, aggregate='sum')

        self.clf = nn.Sequential(
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )


    def forward(self, graph, inputs):
        x = self.conv(graph, inputs)
        x = self.clf(x['tweet'])
        return x

gnn = SAGEClassifier().cuda()
gnn

def forward_pass() -> dict:
    '''A forward pass of the graph neural network.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Set the GNN to training mode
    gnn.train()

    # Get the input features and the output labels
    input_feats = dict(
        article=subgraph.nodes['article'].data['feat'].float().cuda(),
        user=subgraph.nodes['user'].data['feat'].float().cuda(),
        tweet=subgraph.nodes['tweet'].data['feat'].float().cuda(),
        hashtag=subgraph.nodes['hashtag'].data['feat'].float().cuda(),
        image=subgraph.nodes['image'].data['feat'].float().cuda()
        )
    output_labels = subgraph.nodes['tweet'].data['label'].cuda()

    # Forward propagation
    logits = gnn(subgraph, input_feats).squeeze()

    # Compute loss
    loss = F.binary_cross_entropy_with_logits(
        input=logits[train_mask],
        target=output_labels.float()[train_mask]
    )

    # Compute training metrics
    scores = scorer(logits[train_mask].ge(0), output_labels[train_mask])
    misinformation_f1 = scores[0]
    factual_f1 = scores[1]

    return dict(loss=loss, 
                misinformation_f1=misinformation_f1, 
                factual_f1=factual_f1)

def evaluate(split: str) -> dict:
    '''Evaluate the graph neural network.

    Args:
        split (str):
            The split to evaluate the GNN on. Can be 'val' or 'test'.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Get the correct mask, depending on the value of `split`
    mask = val_mask if split == 'val' else test_mask

    gnn.eval()
    with torch.no_grad():

        # Get the input features and the output labels
        input_feats = dict(
            article=subgraph.nodes['article'].data['feat'].float().cuda(),
            user=subgraph.nodes['user'].data['feat'].float().cuda(),
            tweet=subgraph.nodes['tweet'].data['feat'].float().cuda(),
            hashtag=subgraph.nodes['hashtag'].data['feat'].float().cuda(),
            image=subgraph.nodes['image'].data['feat'].float().cuda()
        )
        
        output_labels = subgraph.nodes['tweet'].data['label'].cuda()

        # Forward propagation
        logits = gnn(subgraph, input_feats).squeeze()

        # Compute validation loss
        val_loss = F.binary_cross_entropy_with_logits(
            input=logits[mask],
            target=output_labels.float()[mask]
        ).cpu().item()

        # Compute validation metrics
        scores = scorer(logits[mask].ge(0), output_labels[mask])
        val_misinformation_f1 = scores[0].cpu().item()
        val_factual_f1 = scores[1].cpu().item()

    return dict(loss=val_loss, 
                misinformation_f1=val_misinformation_f1, 
                factual_f1=val_factual_f1)

# Initialise optimiser
opt = optim.AdamW(gnn.parameters(), lr=3e-4)

# Initialise scorer
scorer = tm.classification.f_beta.F1Score(num_classes=2, average='none').cuda()

# Initialise dictionary containing validation scores
val_scores = defaultdict(list)

# Initialise progress bar
epoch_pbar = tqdm(range(1000), desc='Training')

for epoch in epoch_pbar:

    # Reset the gradients
    opt.zero_grad()

    # Forward propagation
    train_results = forward_pass()

    # Backward propagation
    train_results['loss'].backward()

    # Update gradients
    opt.step()

    # Evaluate the model
    val_results = evaluate('val')

    # Store the validation scores
    for metric in ['loss', 'misinformation_f1', 'factual_f1']:
        val_scores[metric].append(val_results[metric])

    # Update progress bar description
    if epoch % 25 == 0 and epoch > 0:
        val_loss = np.mean(val_scores['loss'])
        val_misinformation_f1 = np.mean(val_scores['misinformation_f1'])
        val_factual_f1 = np.mean(val_scores['factual_f1'])
        desc = (f'Training - '
                f'loss {train_results["loss"]:.3f} - '
                f'factual_f1 {train_results["factual_f1"]:.3f} - '
                f'misinfo_f1 {train_results["misinformation_f1"]:.3f} - '
                f'val_loss {val_loss:.3f} - '
                f'val_factual_f1 {val_factual_f1:.3f} - '
                f'val_misinfo_f1 {val_misinformation_f1:.3f}')
        epoch_pbar.set_description(desc)
        val_scores = defaultdict(list)

test_results = evaluate('test')
macro_f1 = np.mean([test_results['factual_f1'],
                    test_results['misinformation_f1']])
print('*** Test results ***')
print(f'Factual F1: {test_results["factual_f1"]:.4f}')
print(f'Misinformation F1: {test_results["misinformation_f1"]:.4f}')
print(f'Macro-average F1: {macro_f1:.4f}')

Training:   0%|          | 0/1000 [00:00<?, ?it/s]

*** Test results ***
Factual F1: 0.2114
Misinformation F1: 0.8783
Macro-average F1: 0.5448


In [ ]:
rel1 = ('article', 'has_article_inv', 'tweet')
rel2 = ('user', 'retweeted', 'tweet')
rel3 = ('user', 'posted', 'tweet')
rel5 = ('image', 'has_image_inv', 'tweet')

subgraph = dgl.edge_type_subgraph(dgl_graph, etypes=[rel1, rel2, rel3, rel5]).to('cuda')
subgraph

train_mask = subgraph.nodes['tweet'].data['train_mask']
val_mask = subgraph.nodes['tweet'].data['val_mask']
test_mask = subgraph.nodes['tweet'].data['test_mask']

class SAGEClassifier(nn.Module):
    def __init__(self, hidden_dim: int = 500):
        super().__init__()
        feats1 = subgraph.nodes['article'].data['feat'].shape[-1]
        feats2 = subgraph.nodes['user'].data['feat'].shape[-1]
        feats4 = subgraph.nodes['image'].data['feat'].shape[-1]
        feats5 = subgraph.nodes['tweet'].data['feat'].shape[-1]
        self.conv = dglnn.HeteroGraphConv({
                'has_article_inv': dglnn.SAGEConv(in_feats=(feats1,feats5), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU()),
                'retweeted': dglnn.SAGEConv(in_feats=(feats2,feats5), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU()),
                'posted': dglnn.SAGEConv(in_feats=(feats2,feats5), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU()),
                'has_image_inv': dglnn.SAGEConv(in_feats=(feats4,feats5), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU())
                }, aggregate='sum')

        self.clf = nn.Sequential(
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )


    def forward(self, graph, inputs):
        x = self.conv(graph, inputs)
        x = self.clf(x['tweet'])
        return x

gnn = SAGEClassifier().cuda()
gnn

def forward_pass() -> dict:
    '''A forward pass of the graph neural network.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Set the GNN to training mode
    gnn.train()

    # Get the input features and the output labels
    input_feats = dict(
        article=subgraph.nodes['article'].data['feat'].float().cuda(),
        user=subgraph.nodes['user'].data['feat'].float().cuda(),
        tweet=subgraph.nodes['tweet'].data['feat'].float().cuda(),
        image=subgraph.nodes['image'].data['feat'].float().cuda()
        )
    output_labels = subgraph.nodes['tweet'].data['label'].cuda()

    # Forward propagation
    logits = gnn(subgraph, input_feats).squeeze()

    # Compute loss
    loss = F.binary_cross_entropy_with_logits(
        input=logits[train_mask],
        target=output_labels.float()[train_mask]
    )

    # Compute training metrics
    scores = scorer(logits[train_mask].ge(0), output_labels[train_mask])
    misinformation_f1 = scores[0]
    factual_f1 = scores[1]

    return dict(loss=loss, 
                misinformation_f1=misinformation_f1, 
                factual_f1=factual_f1)

def evaluate(split: str) -> dict:
    '''Evaluate the graph neural network.

    Args:
        split (str):
            The split to evaluate the GNN on. Can be 'val' or 'test'.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Get the correct mask, depending on the value of `split`
    mask = val_mask if split == 'val' else test_mask

    gnn.eval()
    with torch.no_grad():

        # Get the input features and the output labels
        input_feats = dict(
            article=subgraph.nodes['article'].data['feat'].float().cuda(),
            user=subgraph.nodes['user'].data['feat'].float().cuda(),
            tweet=subgraph.nodes['tweet'].data['feat'].float().cuda(),
            image=subgraph.nodes['image'].data['feat'].float().cuda()
        )
        
        output_labels = subgraph.nodes['tweet'].data['label'].cuda()

        # Forward propagation
        logits = gnn(subgraph, input_feats).squeeze()

        # Compute validation loss
        val_loss = F.binary_cross_entropy_with_logits(
            input=logits[mask],
            target=output_labels.float()[mask]
        ).cpu().item()

        # Compute validation metrics
        scores = scorer(logits[mask].ge(0), output_labels[mask])
        val_misinformation_f1 = scores[0].cpu().item()
        val_factual_f1 = scores[1].cpu().item()

    return dict(loss=val_loss, 
                misinformation_f1=val_misinformation_f1, 
                factual_f1=val_factual_f1)

# Initialise optimiser
opt = optim.AdamW(gnn.parameters(), lr=3e-4)

# Initialise scorer
scorer = tm.classification.f_beta.F1Score(num_classes=2, average='none').cuda()

# Initialise dictionary containing validation scores
val_scores = defaultdict(list)

# Initialise progress bar
epoch_pbar = tqdm(range(1000), desc='Training')

for epoch in epoch_pbar:

    # Reset the gradients
    opt.zero_grad()

    # Forward propagation
    train_results = forward_pass()

    # Backward propagation
    train_results['loss'].backward()

    # Update gradients
    opt.step()

    # Evaluate the model
    val_results = evaluate('val')

    # Store the validation scores
    for metric in ['loss', 'misinformation_f1', 'factual_f1']:
        val_scores[metric].append(val_results[metric])

    # Update progress bar description
    if epoch % 25 == 0 and epoch > 0:
        val_loss = np.mean(val_scores['loss'])
        val_misinformation_f1 = np.mean(val_scores['misinformation_f1'])
        val_factual_f1 = np.mean(val_scores['factual_f1'])
        desc = (f'Training - '
                f'loss {train_results["loss"]:.3f} - '
                f'factual_f1 {train_results["factual_f1"]:.3f} - '
                f'misinfo_f1 {train_results["misinformation_f1"]:.3f} - '
                f'val_loss {val_loss:.3f} - '
                f'val_factual_f1 {val_factual_f1:.3f} - '
                f'val_misinfo_f1 {val_misinformation_f1:.3f}')
        epoch_pbar.set_description(desc)
        val_scores = defaultdict(list)

test_results = evaluate('test')
macro_f1 = np.mean([test_results['factual_f1'],
                    test_results['misinformation_f1']])
print('*** Test results ***')
print(f'Factual F1: {test_results["factual_f1"]:.4f}')
print(f'Misinformation F1: {test_results["misinformation_f1"]:.4f}')
print(f'Macro-average F1: {macro_f1:.4f}')

Training:   0%|          | 0/1000 [00:00<?, ?it/s]

*** Test results ***
Factual F1: 0.1972
Misinformation F1: 0.8535
Macro-average F1: 0.5253


In [ ]:
rel1 = ('article', 'has_article_inv', 'tweet')
rel2 = ('user', 'retweeted', 'tweet')
rel3 = ('user', 'posted', 'tweet')
rel4 = ('hashtag', 'has_hashtag_inv', 'tweet')
rel5 = ('image', 'has_image_inv', 'tweet')
rel6 = ('reply', 'quote_of', 'tweet')

subgraph = dgl.edge_type_subgraph(dgl_graph, etypes=[rel1, rel2, rel3, rel4, rel5, rel6]).to('cuda')
subgraph

train_mask = subgraph.nodes['tweet'].data['train_mask']
val_mask = subgraph.nodes['tweet'].data['val_mask']
test_mask = subgraph.nodes['tweet'].data['test_mask']

class SAGEClassifier(nn.Module):
    def __init__(self, hidden_dim: int = 500):
        super().__init__()
        feats1 = subgraph.nodes['article'].data['feat'].shape[-1]
        feats2 = subgraph.nodes['user'].data['feat'].shape[-1]
        feats3 = subgraph.nodes['hashtag'].data['feat'].shape[-1]
        feats4 = subgraph.nodes['image'].data['feat'].shape[-1]
        feats5 = subgraph.nodes['reply'].data['feat'].shape[-1]
        feats6 = subgraph.nodes['tweet'].data['feat'].shape[-1]
        self.conv = dglnn.HeteroGraphConv({
                'has_article_inv': dglnn.SAGEConv(in_feats=(feats1,feats6), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU()),
                'retweeted': dglnn.SAGEConv(in_feats=(feats2,feats6), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU()),
                'posted': dglnn.SAGEConv(in_feats=(feats2,feats6), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU()),
                'has_hashtag_inv': dglnn.SAGEConv(in_feats=(feats3,feats6), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU()),
                'has_image_inv': dglnn.SAGEConv(in_feats=(feats4,feats6), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU()),
                'quote_of': dglnn.SAGEConv(in_feats=(feats5,feats6), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU())
                }, aggregate='sum')

        self.clf = nn.Sequential(
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )


    def forward(self, graph, inputs):
        x = self.conv(graph, inputs)
        x = self.clf(x['tweet'])
        return x

gnn = SAGEClassifier().cuda()
gnn

def forward_pass() -> dict:
    '''A forward pass of the graph neural network.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Set the GNN to training mode
    gnn.train()

    # Get the input features and the output labels
    input_feats = dict(
        article=subgraph.nodes['article'].data['feat'].float().cuda(),
        user=subgraph.nodes['user'].data['feat'].float().cuda(),
        tweet=subgraph.nodes['tweet'].data['feat'].float().cuda(),
        hashtag=subgraph.nodes['hashtag'].data['feat'].float().cuda(),
        image=subgraph.nodes['image'].data['feat'].float().cuda(),
        reply=subgraph.nodes['reply'].data['feat'].float().cuda()
        )
    output_labels = subgraph.nodes['tweet'].data['label'].cuda()

    # Forward propagation
    logits = gnn(subgraph, input_feats).squeeze()

    # Compute loss
    loss = F.binary_cross_entropy_with_logits(
        input=logits[train_mask],
        target=output_labels.float()[train_mask]
    )

    # Compute training metrics
    scores = scorer(logits[train_mask].ge(0), output_labels[train_mask])
    misinformation_f1 = scores[0]
    factual_f1 = scores[1]

    return dict(loss=loss, 
                misinformation_f1=misinformation_f1, 
                factual_f1=factual_f1)

def evaluate(split: str) -> dict:
    '''Evaluate the graph neural network.

    Args:
        split (str):
            The split to evaluate the GNN on. Can be 'val' or 'test'.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Get the correct mask, depending on the value of `split`
    mask = val_mask if split == 'val' else test_mask

    gnn.eval()
    with torch.no_grad():

        # Get the input features and the output labels
        input_feats = dict(
            article=subgraph.nodes['article'].data['feat'].float().cuda(),
            user=subgraph.nodes['user'].data['feat'].float().cuda(),
            tweet=subgraph.nodes['tweet'].data['feat'].float().cuda(),
            hashtag=subgraph.nodes['hashtag'].data['feat'].float().cuda(),
            image=subgraph.nodes['image'].data['feat'].float().cuda(),
            reply=subgraph.nodes['reply'].data['feat'].float().cuda()
        )
        
        output_labels = subgraph.nodes['tweet'].data['label'].cuda()

        # Forward propagation
        logits = gnn(subgraph, input_feats).squeeze()

        # Compute validation loss
        val_loss = F.binary_cross_entropy_with_logits(
            input=logits[mask],
            target=output_labels.float()[mask]
        ).cpu().item()

        # Compute validation metrics
        scores = scorer(logits[mask].ge(0), output_labels[mask])
        val_misinformation_f1 = scores[0].cpu().item()
        val_factual_f1 = scores[1].cpu().item()

    return dict(loss=val_loss, 
                misinformation_f1=val_misinformation_f1, 
                factual_f1=val_factual_f1)

# Initialise optimiser
opt = optim.AdamW(gnn.parameters(), lr=3e-4)

# Initialise scorer
scorer = tm.classification.f_beta.F1Score(num_classes=2, average='none').cuda()

# Initialise dictionary containing validation scores
val_scores = defaultdict(list)

# Initialise progress bar
epoch_pbar = tqdm(range(1000), desc='Training')

for epoch in epoch_pbar:

    # Reset the gradients
    opt.zero_grad()

    # Forward propagation
    train_results = forward_pass()

    # Backward propagation
    train_results['loss'].backward()

    # Update gradients
    opt.step()

    # Evaluate the model
    val_results = evaluate('val')

    # Store the validation scores
    for metric in ['loss', 'misinformation_f1', 'factual_f1']:
        val_scores[metric].append(val_results[metric])

    # Update progress bar description
    if epoch % 25 == 0 and epoch > 0:
        val_loss = np.mean(val_scores['loss'])
        val_misinformation_f1 = np.mean(val_scores['misinformation_f1'])
        val_factual_f1 = np.mean(val_scores['factual_f1'])
        desc = (f'Training - '
                f'loss {train_results["loss"]:.3f} - '
                f'factual_f1 {train_results["factual_f1"]:.3f} - '
                f'misinfo_f1 {train_results["misinformation_f1"]:.3f} - '
                f'val_loss {val_loss:.3f} - '
                f'val_factual_f1 {val_factual_f1:.3f} - '
                f'val_misinfo_f1 {val_misinformation_f1:.3f}')
        epoch_pbar.set_description(desc)
        val_scores = defaultdict(list)

test_results = evaluate('test')
macro_f1 = np.mean([test_results['factual_f1'],
                    test_results['misinformation_f1']])
print('*** Test results ***')
print(f'Factual F1: {test_results["factual_f1"]:.4f}')
print(f'Misinformation F1: {test_results["misinformation_f1"]:.4f}')
print(f'Macro-average F1: {macro_f1:.4f}')

Training:   0%|          | 0/1000 [00:00<?, ?it/s]

*** Test results ***
Factual F1: 0.1951
Misinformation F1: 0.8758
Macro-average F1: 0.5355


In [ ]:
rel1 = ('article', 'has_article_inv', 'tweet')
rel2 = ('user', 'retweeted', 'tweet')
rel3 = ('user', 'posted', 'tweet')
rel4 = ('hashtag', 'has_hashtag_inv', 'tweet')
rel5 = ('image', 'has_image_inv', 'tweet')
rel6 = ('reply', 'quote_of', 'tweet')
rel7 = ('reply', 'reply_to', 'tweet')

subgraph = dgl.edge_type_subgraph(dgl_graph, etypes=[rel1, rel2, rel3, rel4, rel5, rel6, rel7]).to('cuda')
subgraph

train_mask = subgraph.nodes['tweet'].data['train_mask']
val_mask = subgraph.nodes['tweet'].data['val_mask']
test_mask = subgraph.nodes['tweet'].data['test_mask']

class SAGEClassifier(nn.Module):
    def __init__(self, hidden_dim: int = 500):
        super().__init__()
        feats1 = subgraph.nodes['article'].data['feat'].shape[-1]
        feats2 = subgraph.nodes['user'].data['feat'].shape[-1]
        feats3 = subgraph.nodes['hashtag'].data['feat'].shape[-1]
        feats4 = subgraph.nodes['image'].data['feat'].shape[-1]
        feats5 = subgraph.nodes['reply'].data['feat'].shape[-1]
        feats6 = subgraph.nodes['tweet'].data['feat'].shape[-1]
        self.conv = dglnn.HeteroGraphConv({
                'has_article_inv': dglnn.SAGEConv(in_feats=(feats1,feats6), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU()),
                'retweeted': dglnn.SAGEConv(in_feats=(feats2,feats6), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU()),
                'posted': dglnn.SAGEConv(in_feats=(feats2,feats6), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU()),
                'has_hashtag_inv': dglnn.SAGEConv(in_feats=(feats3,feats6), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU()),
                'has_image_inv': dglnn.SAGEConv(in_feats=(feats4,feats6), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU()),
                'quote_of': dglnn.SAGEConv(in_feats=(feats5,feats6), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU()),
                'reply_to': dglnn.SAGEConv(in_feats=(feats5,feats6), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU())
                }, aggregate='sum')

        self.clf = nn.Sequential(
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )


    def forward(self, graph, inputs):
        x = self.conv(graph, inputs)
        x = self.clf(x['tweet'])
        return x

gnn = SAGEClassifier().cuda()
gnn

def forward_pass() -> dict:
    '''A forward pass of the graph neural network.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Set the GNN to training mode
    gnn.train()

    # Get the input features and the output labels
    input_feats = dict(
        article=subgraph.nodes['article'].data['feat'].float().cuda(),
        user=subgraph.nodes['user'].data['feat'].float().cuda(),
        tweet=subgraph.nodes['tweet'].data['feat'].float().cuda(),
        hashtag=subgraph.nodes['hashtag'].data['feat'].float().cuda(),
        image=subgraph.nodes['image'].data['feat'].float().cuda(),
        reply=subgraph.nodes['reply'].data['feat'].float().cuda()
        )
    output_labels = subgraph.nodes['tweet'].data['label'].cuda()

    # Forward propagation
    logits = gnn(subgraph, input_feats).squeeze()

    # Compute loss
    loss = F.binary_cross_entropy_with_logits(
        input=logits[train_mask],
        target=output_labels.float()[train_mask]
    )

    # Compute training metrics
    scores = scorer(logits[train_mask].ge(0), output_labels[train_mask])
    misinformation_f1 = scores[0]
    factual_f1 = scores[1]

    return dict(loss=loss, 
                misinformation_f1=misinformation_f1, 
                factual_f1=factual_f1)

def evaluate(split: str) -> dict:
    '''Evaluate the graph neural network.

    Args:
        split (str):
            The split to evaluate the GNN on. Can be 'val' or 'test'.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Get the correct mask, depending on the value of `split`
    mask = val_mask if split == 'val' else test_mask

    gnn.eval()
    with torch.no_grad():

        # Get the input features and the output labels
        input_feats = dict(
            article=subgraph.nodes['article'].data['feat'].float().cuda(),
            user=subgraph.nodes['user'].data['feat'].float().cuda(),
            tweet=subgraph.nodes['tweet'].data['feat'].float().cuda(),
            hashtag=subgraph.nodes['hashtag'].data['feat'].float().cuda(),
            image=subgraph.nodes['image'].data['feat'].float().cuda(),
            reply=subgraph.nodes['reply'].data['feat'].float().cuda()
        )
        
        output_labels = subgraph.nodes['tweet'].data['label'].cuda()

        # Forward propagation
        logits = gnn(subgraph, input_feats).squeeze()

        # Compute validation loss
        val_loss = F.binary_cross_entropy_with_logits(
            input=logits[mask],
            target=output_labels.float()[mask]
        ).cpu().item()

        # Compute validation metrics
        scores = scorer(logits[mask].ge(0), output_labels[mask])
        val_misinformation_f1 = scores[0].cpu().item()
        val_factual_f1 = scores[1].cpu().item()

    return dict(loss=val_loss, 
                misinformation_f1=val_misinformation_f1, 
                factual_f1=val_factual_f1)

# Initialise optimiser
opt = optim.AdamW(gnn.parameters(), lr=3e-4)

# Initialise scorer
scorer = tm.classification.f_beta.F1Score(num_classes=2, average='none').cuda()

# Initialise dictionary containing validation scores
val_scores = defaultdict(list)

# Initialise progress bar
epoch_pbar = tqdm(range(1000), desc='Training')

for epoch in epoch_pbar:

    # Reset the gradients
    opt.zero_grad()

    # Forward propagation
    train_results = forward_pass()

    # Backward propagation
    train_results['loss'].backward()

    # Update gradients
    opt.step()

    # Evaluate the model
    val_results = evaluate('val')

    # Store the validation scores
    for metric in ['loss', 'misinformation_f1', 'factual_f1']:
        val_scores[metric].append(val_results[metric])

    # Update progress bar description
    if epoch % 25 == 0 and epoch > 0:
        val_loss = np.mean(val_scores['loss'])
        val_misinformation_f1 = np.mean(val_scores['misinformation_f1'])
        val_factual_f1 = np.mean(val_scores['factual_f1'])
        desc = (f'Training - '
                f'loss {train_results["loss"]:.3f} - '
                f'factual_f1 {train_results["factual_f1"]:.3f} - '
                f'misinfo_f1 {train_results["misinformation_f1"]:.3f} - '
                f'val_loss {val_loss:.3f} - '
                f'val_factual_f1 {val_factual_f1:.3f} - '
                f'val_misinfo_f1 {val_misinformation_f1:.3f}')
        epoch_pbar.set_description(desc)
        val_scores = defaultdict(list)

test_results = evaluate('test')
macro_f1 = np.mean([test_results['factual_f1'],
                    test_results['misinformation_f1']])
print('*** Test results ***')
print(f'Factual F1: {test_results["factual_f1"]:.4f}')
print(f'Misinformation F1: {test_results["misinformation_f1"]:.4f}')
print(f'Macro-average F1: {macro_f1:.4f}')

Training:   0%|          | 0/1000 [00:00<?, ?it/s]

*** Test results ***
Factual F1: 0.1549
Misinformation F1: 0.8458
Macro-average F1: 0.5003


Okay, the results are looking alright! For a final evaluation of the model, let's run our `evaluate` function on the test split:

And that's it, we got a graph model! Note that there's not a big difference between this and the text-only model from above. But we're also not adding a lot more, only the features of the author of the tweet. By adding more nodes and processing them in a neat novel way (this is where you come in!), we should be able to push the performance even more.

## 'reply', 'reply_to', 'tweet'

In [ ]:
rel = ('reply', 'reply_to', 'tweet')
subgraph = dgl.edge_type_subgraph(dgl_graph, etypes=[rel]).to('cuda')
subgraph

train_mask = subgraph.nodes['tweet'].data['train_mask']
val_mask = subgraph.nodes['tweet'].data['val_mask']
test_mask = subgraph.nodes['tweet'].data['test_mask']

class SAGEClassifier(nn.Module):
    def __init__(self, hidden_dim: int = 500):
        super().__init__()
        feats1 = subgraph.nodes[rel[0]].data['feat'].shape[-1]
        feats2 = subgraph.nodes[rel[2]].data['feat'].shape[-1]
        self.conv = dglnn.SAGEConv(in_feats=(feats1, feats2), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU())
        self.clf = nn.Sequential(
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )


    def forward(self, graph, x):
        x = self.conv(graph, (x['user'], x['tweet']))
        x = self.clf(x)
        return x

gnn = SAGEClassifier().cuda()
gnn

def forward_pass() -> dict:
    '''A forward pass of the graph neural network.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Set the GNN to training mode
    gnn.train()

    # Get the input features and the output labels
    input_feats = dict(
        user=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
        tweet=subgraph.nodes[rel[2]].data['feat'].float().cuda()
    )
    output_labels = subgraph.nodes['tweet'].data['label'].cuda()

    # Forward propagation
    logits = gnn(subgraph, input_feats).squeeze()

    # Compute loss
    loss = F.binary_cross_entropy_with_logits(
        input=logits[train_mask],
        target=output_labels.float()[train_mask]
    )

    # Compute training metrics
    scores = scorer(logits[train_mask].ge(0), output_labels[train_mask])
    misinformation_f1 = scores[0]
    factual_f1 = scores[1]

    return dict(loss=loss, 
                misinformation_f1=misinformation_f1, 
                factual_f1=factual_f1)

def evaluate(split: str) -> dict:
    '''Evaluate the graph neural network.

    Args:
        split (str):
            The split to evaluate the GNN on. Can be 'val' or 'test'.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Get the correct mask, depending on the value of `split`
    mask = val_mask if split == 'val' else test_mask

    gnn.eval()
    with torch.no_grad():

        # Get the input features and the output labels
        input_feats = dict(
            user=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
            tweet=subgraph.nodes[rel[2]].data['feat'].float().cuda()
        )
        output_labels = subgraph.nodes['tweet'].data['label'].cuda()

        # Forward propagation
        logits = gnn(subgraph, input_feats).squeeze()

        # Compute validation loss
        val_loss = F.binary_cross_entropy_with_logits(
            input=logits[mask],
            target=output_labels.float()[mask]
        ).cpu().item()

        # Compute validation metrics
        scores = scorer(logits[mask].ge(0), output_labels[mask])
        val_misinformation_f1 = scores[0].cpu().item()
        val_factual_f1 = scores[1].cpu().item()

    return dict(loss=val_loss, 
                misinformation_f1=val_misinformation_f1, 
                factual_f1=val_factual_f1)

# Initialise optimiser
opt = optim.AdamW(gnn.parameters(), lr=3e-4)

# Initialise scorer
scorer = tm.classification.f_beta.F1Score(num_classes=2, average='none').cuda()

# Initialise dictionary containing validation scores
val_scores = defaultdict(list)

# Initialise progress bar
epoch_pbar = tqdm(range(1000), desc='Training')

for epoch in epoch_pbar:

    # Reset the gradients
    opt.zero_grad()

    # Forward propagation
    train_results = forward_pass()

    # Backward propagation
    train_results['loss'].backward()

    # Update gradients
    opt.step()

    # Evaluate the model
    val_results = evaluate('val')

    # Store the validation scores
    for metric in ['loss', 'misinformation_f1', 'factual_f1']:
        val_scores[metric].append(val_results[metric])

    # Update progress bar description
    if epoch % 25 == 0 and epoch > 0:
        val_loss = np.mean(val_scores['loss'])
        val_misinformation_f1 = np.mean(val_scores['misinformation_f1'])
        val_factual_f1 = np.mean(val_scores['factual_f1'])
        desc = (f'Training - '
                f'loss {train_results["loss"]:.3f} - '
                f'factual_f1 {train_results["factual_f1"]:.3f} - '
                f'misinfo_f1 {train_results["misinformation_f1"]:.3f} - '
                f'val_loss {val_loss:.3f} - '
                f'val_factual_f1 {val_factual_f1:.3f} - '
                f'val_misinfo_f1 {val_misinformation_f1:.3f}')
        epoch_pbar.set_description(desc)
        val_scores = defaultdict(list)

test_results = evaluate('test')
macro_f1 = np.mean([test_results['factual_f1'],
                    test_results['misinformation_f1']])
print('*** Test results ***')
print(f'Factual F1: {test_results["factual_f1"]:.4f}')
print(f'Misinformation F1: {test_results["misinformation_f1"]:.4f}')
print(f'Macro-average F1: {macro_f1:.4f}')

Training:   0%|          | 0/1000 [00:00<?, ?it/s]

*** Test results ***
Factual F1: 0.1644
Misinformation F1: 0.9280
Macro-average F1: 0.5462


## 'claim', 'discusses_inv', 'tweet'


In [ ]:
rel = ('claim', 'discusses_inv', 'tweet')
subgraph = dgl.edge_type_subgraph(dgl_graph, etypes=[rel]).to('cuda')
subgraph

train_mask = subgraph.nodes['tweet'].data['train_mask']
val_mask = subgraph.nodes['tweet'].data['val_mask']
test_mask = subgraph.nodes['tweet'].data['test_mask']

class SAGEClassifier(nn.Module):
    def __init__(self, hidden_dim: int = 500):
        super().__init__()
        feats1 = subgraph.nodes[rel[0]].data['feat'].shape[-1]
        feats2 = subgraph.nodes[rel[2]].data['feat'].shape[-1]
        self.conv = dglnn.SAGEConv(in_feats=(feats1, feats2), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU())
        self.clf = nn.Sequential(
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )


    def forward(self, graph, x):
        x = self.conv(graph, (x['p1'], x['p2']))
        x = self.clf(x)
        return x

gnn = SAGEClassifier().cuda()
gnn

def forward_pass() -> dict:
    '''A forward pass of the graph neural network.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Set the GNN to training mode
    gnn.train()

    # Get the input features and the output labels
    input_feats = dict(
        p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
        p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
    )
    output_labels = subgraph.nodes['tweet'].data['label'].cuda()

    # Forward propagation
    logits = gnn(subgraph, input_feats).squeeze()

    # Compute loss
    loss = F.binary_cross_entropy_with_logits(
        input=logits[train_mask],
        target=output_labels.float()[train_mask]
    )

    # Compute training metrics
    scores = scorer(logits[train_mask].ge(0), output_labels[train_mask])
    misinformation_f1 = scores[0]
    factual_f1 = scores[1]

    return dict(loss=loss, 
                misinformation_f1=misinformation_f1, 
                factual_f1=factual_f1)

def evaluate(split: str) -> dict:
    '''Evaluate the graph neural network.

    Args:
        split (str):
            The split to evaluate the GNN on. Can be 'val' or 'test'.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Get the correct mask, depending on the value of `split`
    mask = val_mask if split == 'val' else test_mask

    gnn.eval()
    with torch.no_grad():

        # Get the input features and the output labels
        input_feats = dict(
            p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
            p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
        )
        output_labels = subgraph.nodes['tweet'].data['label'].cuda()

        # Forward propagation
        logits = gnn(subgraph, input_feats).squeeze()

        # Compute validation loss
        val_loss = F.binary_cross_entropy_with_logits(
            input=logits[mask],
            target=output_labels.float()[mask]
        ).cpu().item()

        # Compute validation metrics
        scores = scorer(logits[mask].ge(0), output_labels[mask])
        val_misinformation_f1 = scores[0].cpu().item()
        val_factual_f1 = scores[1].cpu().item()

    return dict(loss=val_loss, 
                misinformation_f1=val_misinformation_f1, 
                factual_f1=val_factual_f1)

# Initialise optimiser
opt = optim.AdamW(gnn.parameters(), lr=3e-4)

# Initialise scorer
scorer = tm.classification.f_beta.F1Score(num_classes=2, average='none').cuda()

# Initialise dictionary containing validation scores
val_scores = defaultdict(list)

# Initialise progress bar
epoch_pbar = tqdm(range(1000), desc='Training')

for epoch in epoch_pbar:

    # Reset the gradients
    opt.zero_grad()

    # Forward propagation
    train_results = forward_pass()

    # Backward propagation
    train_results['loss'].backward()

    # Update gradients
    opt.step()

    # Evaluate the model
    val_results = evaluate('val')

    # Store the validation scores
    for metric in ['loss', 'misinformation_f1', 'factual_f1']:
        val_scores[metric].append(val_results[metric])

    # Update progress bar description
    if epoch % 25 == 0 and epoch > 0:
        val_loss = np.mean(val_scores['loss'])
        val_misinformation_f1 = np.mean(val_scores['misinformation_f1'])
        val_factual_f1 = np.mean(val_scores['factual_f1'])
        desc = (f'Training - '
                f'loss {train_results["loss"]:.3f} - '
                f'factual_f1 {train_results["factual_f1"]:.3f} - '
                f'misinfo_f1 {train_results["misinformation_f1"]:.3f} - '
                f'val_loss {val_loss:.3f} - '
                f'val_factual_f1 {val_factual_f1:.3f} - '
                f'val_misinfo_f1 {val_misinformation_f1:.3f}')
        epoch_pbar.set_description(desc)
        val_scores = defaultdict(list)

test_results = evaluate('test')
macro_f1 = np.mean([test_results['factual_f1'],
                    test_results['misinformation_f1']])
print('*** Test results ***')
print(f'Factual F1: {test_results["factual_f1"]:.4f}')
print(f'Misinformation F1: {test_results["misinformation_f1"]:.4f}')
print(f'Macro-average F1: {macro_f1:.4f}')

Training:   0%|          | 0/1000 [00:00<?, ?it/s]

*** Test results ***
Factual F1: 0.1044
Misinformation F1: 0.4137
Macro-average F1: 0.2590


## 'reply', 'quote_of', 'tweet'

In [ ]:
rel = ('reply', 'quote_of', 'tweet')
subgraph = dgl.edge_type_subgraph(dgl_graph, etypes=[rel]).to('cuda')
subgraph

train_mask = subgraph.nodes['tweet'].data['train_mask']
val_mask = subgraph.nodes['tweet'].data['val_mask']
test_mask = subgraph.nodes['tweet'].data['test_mask']

class SAGEClassifier(nn.Module):
    def __init__(self, hidden_dim: int = 500):
        super().__init__()
        feats1 = subgraph.nodes[rel[0]].data['feat'].shape[-1]
        feats2 = subgraph.nodes[rel[2]].data['feat'].shape[-1]
        self.conv = dglnn.SAGEConv(in_feats=(feats1, feats2), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU())
        self.clf = nn.Sequential(
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )


    def forward(self, graph, x):
        x = self.conv(graph, (x['p1'], x['p2']))
        x = self.clf(x)
        return x

gnn = SAGEClassifier().cuda()
gnn

def forward_pass() -> dict:
    '''A forward pass of the graph neural network.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Set the GNN to training mode
    gnn.train()

    # Get the input features and the output labels
    input_feats = dict(
        p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
        p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
    )
    output_labels = subgraph.nodes['tweet'].data['label'].cuda()

    # Forward propagation
    logits = gnn(subgraph, input_feats).squeeze()

    # Compute loss
    loss = F.binary_cross_entropy_with_logits(
        input=logits[train_mask],
        target=output_labels.float()[train_mask]
    )

    # Compute training metrics
    scores = scorer(logits[train_mask].ge(0), output_labels[train_mask])
    misinformation_f1 = scores[0]
    factual_f1 = scores[1]

    return dict(loss=loss, 
                misinformation_f1=misinformation_f1, 
                factual_f1=factual_f1)

def evaluate(split: str) -> dict:
    '''Evaluate the graph neural network.

    Args:
        split (str):
            The split to evaluate the GNN on. Can be 'val' or 'test'.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Get the correct mask, depending on the value of `split`
    mask = val_mask if split == 'val' else test_mask

    gnn.eval()
    with torch.no_grad():

        # Get the input features and the output labels
        input_feats = dict(
            p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
            p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
        )
        output_labels = subgraph.nodes['tweet'].data['label'].cuda()

        # Forward propagation
        logits = gnn(subgraph, input_feats).squeeze()

        # Compute validation loss
        val_loss = F.binary_cross_entropy_with_logits(
            input=logits[mask],
            target=output_labels.float()[mask]
        ).cpu().item()

        # Compute validation metrics
        scores = scorer(logits[mask].ge(0), output_labels[mask])
        val_misinformation_f1 = scores[0].cpu().item()
        val_factual_f1 = scores[1].cpu().item()

    return dict(loss=val_loss, 
                misinformation_f1=val_misinformation_f1, 
                factual_f1=val_factual_f1)

# Initialise optimiser
opt = optim.AdamW(gnn.parameters(), lr=3e-4)

# Initialise scorer
scorer = tm.classification.f_beta.F1Score(num_classes=2, average='none').cuda()

# Initialise dictionary containing validation scores
val_scores = defaultdict(list)

# Initialise progress bar
epoch_pbar = tqdm(range(1000), desc='Training')

for epoch in epoch_pbar:

    # Reset the gradients
    opt.zero_grad()

    # Forward propagation
    train_results = forward_pass()

    # Backward propagation
    train_results['loss'].backward()

    # Update gradients
    opt.step()

    # Evaluate the model
    val_results = evaluate('val')

    # Store the validation scores
    for metric in ['loss', 'misinformation_f1', 'factual_f1']:
        val_scores[metric].append(val_results[metric])

    # Update progress bar description
    if epoch % 25 == 0 and epoch > 0:
        val_loss = np.mean(val_scores['loss'])
        val_misinformation_f1 = np.mean(val_scores['misinformation_f1'])
        val_factual_f1 = np.mean(val_scores['factual_f1'])
        desc = (f'Training - '
                f'loss {train_results["loss"]:.3f} - '
                f'factual_f1 {train_results["factual_f1"]:.3f} - '
                f'misinfo_f1 {train_results["misinformation_f1"]:.3f} - '
                f'val_loss {val_loss:.3f} - '
                f'val_factual_f1 {val_factual_f1:.3f} - '
                f'val_misinfo_f1 {val_misinformation_f1:.3f}')
        epoch_pbar.set_description(desc)
        val_scores = defaultdict(list)

test_results = evaluate('test')
macro_f1 = np.mean([test_results['factual_f1'],
                    test_results['misinformation_f1']])
print('*** Test results ***')
print(f'Factual F1: {test_results["factual_f1"]:.4f}')
print(f'Misinformation F1: {test_results["misinformation_f1"]:.4f}')
print(f'Macro-average F1: {macro_f1:.4f}')

Training:   0%|          | 0/1000 [00:00<?, ?it/s]

*** Test results ***
Factual F1: 0.1791
Misinformation F1: 0.9355
Macro-average F1: 0.5573


## 'article', 'has_article_inv', 'tweet'

In [ ]:
rel = ('article', 'has_article_inv', 'tweet')
subgraph = dgl.edge_type_subgraph(dgl_graph, etypes=[rel]).to('cuda')
subgraph

train_mask = subgraph.nodes['tweet'].data['train_mask']
val_mask = subgraph.nodes['tweet'].data['val_mask']
test_mask = subgraph.nodes['tweet'].data['test_mask']

class SAGEClassifier(nn.Module):
    def __init__(self, hidden_dim: int = 500):
        super().__init__()
        feats1 = subgraph.nodes[rel[0]].data['feat'].shape[-1]
        feats2 = subgraph.nodes[rel[2]].data['feat'].shape[-1]
        self.conv = dglnn.SAGEConv(in_feats=(feats1, feats2), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU())
        self.clf = nn.Sequential(
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )


    def forward(self, graph, x):
        x = self.conv(graph, (x['p1'], x['p2']))
        x = self.clf(x)
        return x

gnn = SAGEClassifier().cuda()
gnn

def forward_pass() -> dict:
    '''A forward pass of the graph neural network.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Set the GNN to training mode
    gnn.train()

    # Get the input features and the output labels
    input_feats = dict(
        p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
        p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
    )
    output_labels = subgraph.nodes['tweet'].data['label'].cuda()

    # Forward propagation
    logits = gnn(subgraph, input_feats).squeeze()

    # Compute loss
    loss = F.binary_cross_entropy_with_logits(
        input=logits[train_mask],
        target=output_labels.float()[train_mask]
    )

    # Compute training metrics
    scores = scorer(logits[train_mask].ge(0), output_labels[train_mask])
    misinformation_f1 = scores[0]
    factual_f1 = scores[1]

    return dict(loss=loss, 
                misinformation_f1=misinformation_f1, 
                factual_f1=factual_f1)

def evaluate(split: str) -> dict:
    '''Evaluate the graph neural network.

    Args:
        split (str):
            The split to evaluate the GNN on. Can be 'val' or 'test'.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Get the correct mask, depending on the value of `split`
    mask = val_mask if split == 'val' else test_mask

    gnn.eval()
    with torch.no_grad():

        # Get the input features and the output labels
        input_feats = dict(
            p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
            p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
        )
        output_labels = subgraph.nodes['tweet'].data['label'].cuda()

        # Forward propagation
        logits = gnn(subgraph, input_feats).squeeze()

        # Compute validation loss
        val_loss = F.binary_cross_entropy_with_logits(
            input=logits[mask],
            target=output_labels.float()[mask]
        ).cpu().item()

        # Compute validation metrics
        scores = scorer(logits[mask].ge(0), output_labels[mask])
        val_misinformation_f1 = scores[0].cpu().item()
        val_factual_f1 = scores[1].cpu().item()

    return dict(loss=val_loss, 
                misinformation_f1=val_misinformation_f1, 
                factual_f1=val_factual_f1)

# Initialise optimiser
opt = optim.AdamW(gnn.parameters(), lr=3e-4)

# Initialise scorer
scorer = tm.classification.f_beta.F1Score(num_classes=2, average='none').cuda()

# Initialise dictionary containing validation scores
val_scores = defaultdict(list)

# Initialise progress bar
epoch_pbar = tqdm(range(1000), desc='Training')

for epoch in epoch_pbar:

    # Reset the gradients
    opt.zero_grad()

    # Forward propagation
    train_results = forward_pass()

    # Backward propagation
    train_results['loss'].backward()

    # Update gradients
    opt.step()

    # Evaluate the model
    val_results = evaluate('val')

    # Store the validation scores
    for metric in ['loss', 'misinformation_f1', 'factual_f1']:
        val_scores[metric].append(val_results[metric])

    # Update progress bar description
    if epoch % 25 == 0 and epoch > 0:
        val_loss = np.mean(val_scores['loss'])
        val_misinformation_f1 = np.mean(val_scores['misinformation_f1'])
        val_factual_f1 = np.mean(val_scores['factual_f1'])
        desc = (f'Training - '
                f'loss {train_results["loss"]:.3f} - '
                f'factual_f1 {train_results["factual_f1"]:.3f} - '
                f'misinfo_f1 {train_results["misinformation_f1"]:.3f} - '
                f'val_loss {val_loss:.3f} - '
                f'val_factual_f1 {val_factual_f1:.3f} - '
                f'val_misinfo_f1 {val_misinformation_f1:.3f}')
        epoch_pbar.set_description(desc)
        val_scores = defaultdict(list)

test_results = evaluate('test')
macro_f1 = np.mean([test_results['factual_f1'],
                    test_results['misinformation_f1']])
print('*** Test results ***')
print(f'Factual F1: {test_results["factual_f1"]:.4f}')
print(f'Misinformation F1: {test_results["misinformation_f1"]:.4f}')
print(f'Macro-average F1: {macro_f1:.4f}')

Training:   0%|          | 0/1000 [00:00<?, ?it/s]

*** Test results ***
Factual F1: 0.2703
Misinformation F1: 0.9694
Macro-average F1: 0.6198


## 'hashtag', 'has_hashtag_inv', 'tweet'

In [ ]:
rel = ('hashtag', 'has_hashtag_inv', 'tweet')
subgraph = dgl.edge_type_subgraph(dgl_graph, etypes=[rel]).to('cuda')
subgraph

train_mask = subgraph.nodes['tweet'].data['train_mask']
val_mask = subgraph.nodes['tweet'].data['val_mask']
test_mask = subgraph.nodes['tweet'].data['test_mask']

class SAGEClassifier(nn.Module):
    def __init__(self, hidden_dim: int = 500):
        super().__init__()
        feats1 = subgraph.nodes[rel[0]].data['feat'].shape[-1]
        feats2 = subgraph.nodes[rel[2]].data['feat'].shape[-1]
        self.conv = dglnn.SAGEConv(in_feats=(feats1, feats2), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU())
        self.clf = nn.Sequential(
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )


    def forward(self, graph, x):
        x = self.conv(graph, (x['p1'], x['p2']))
        x = self.clf(x)
        return x

gnn = SAGEClassifier().cuda()
gnn

def forward_pass() -> dict:
    '''A forward pass of the graph neural network.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Set the GNN to training mode
    gnn.train()

    # Get the input features and the output labels
    input_feats = dict(
        p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
        p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
    )
    output_labels = subgraph.nodes['tweet'].data['label'].cuda()

    # Forward propagation
    logits = gnn(subgraph, input_feats).squeeze()

    # Compute loss
    loss = F.binary_cross_entropy_with_logits(
        input=logits[train_mask],
        target=output_labels.float()[train_mask]
    )

    # Compute training metrics
    scores = scorer(logits[train_mask].ge(0), output_labels[train_mask])
    misinformation_f1 = scores[0]
    factual_f1 = scores[1]

    return dict(loss=loss, 
                misinformation_f1=misinformation_f1, 
                factual_f1=factual_f1)

def evaluate(split: str) -> dict:
    '''Evaluate the graph neural network.

    Args:
        split (str):
            The split to evaluate the GNN on. Can be 'val' or 'test'.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Get the correct mask, depending on the value of `split`
    mask = val_mask if split == 'val' else test_mask

    gnn.eval()
    with torch.no_grad():

        # Get the input features and the output labels
        input_feats = dict(
            p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
            p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
        )
        output_labels = subgraph.nodes['tweet'].data['label'].cuda()

        # Forward propagation
        logits = gnn(subgraph, input_feats).squeeze()

        # Compute validation loss
        val_loss = F.binary_cross_entropy_with_logits(
            input=logits[mask],
            target=output_labels.float()[mask]
        ).cpu().item()

        # Compute validation metrics
        scores = scorer(logits[mask].ge(0), output_labels[mask])
        val_misinformation_f1 = scores[0].cpu().item()
        val_factual_f1 = scores[1].cpu().item()

    return dict(loss=val_loss, 
                misinformation_f1=val_misinformation_f1, 
                factual_f1=val_factual_f1)

# Initialise optimiser
opt = optim.AdamW(gnn.parameters(), lr=3e-4)

# Initialise scorer
scorer = tm.classification.f_beta.F1Score(num_classes=2, average='none').cuda()

# Initialise dictionary containing validation scores
val_scores = defaultdict(list)

# Initialise progress bar
epoch_pbar = tqdm(range(1000), desc='Training')

for epoch in epoch_pbar:

    # Reset the gradients
    opt.zero_grad()

    # Forward propagation
    train_results = forward_pass()

    # Backward propagation
    train_results['loss'].backward()

    # Update gradients
    opt.step()

    # Evaluate the model
    val_results = evaluate('val')

    # Store the validation scores
    for metric in ['loss', 'misinformation_f1', 'factual_f1']:
        val_scores[metric].append(val_results[metric])

    # Update progress bar description
    if epoch % 25 == 0 and epoch > 0:
        val_loss = np.mean(val_scores['loss'])
        val_misinformation_f1 = np.mean(val_scores['misinformation_f1'])
        val_factual_f1 = np.mean(val_scores['factual_f1'])
        desc = (f'Training - '
                f'loss {train_results["loss"]:.3f} - '
                f'factual_f1 {train_results["factual_f1"]:.3f} - '
                f'misinfo_f1 {train_results["misinformation_f1"]:.3f} - '
                f'val_loss {val_loss:.3f} - '
                f'val_factual_f1 {val_factual_f1:.3f} - '
                f'val_misinfo_f1 {val_misinformation_f1:.3f}')
        epoch_pbar.set_description(desc)
        val_scores = defaultdict(list)

test_results = evaluate('test')
macro_f1 = np.mean([test_results['factual_f1'],
                    test_results['misinformation_f1']])
print('*** Test results ***')
print(f'Factual F1: {test_results["factual_f1"]:.4f}')
print(f'Misinformation F1: {test_results["misinformation_f1"]:.4f}')
print(f'Macro-average F1: {macro_f1:.4f}')

Training:   0%|          | 0/1000 [00:00<?, ?it/s]

*** Test results ***
Factual F1: 0.1250
Misinformation F1: 0.9685
Macro-average F1: 0.5467


## 'image', 'has_image_inv', 'tweet'

In [ ]:
rel = ('image', 'has_image_inv', 'tweet')
subgraph = dgl.edge_type_subgraph(dgl_graph, etypes=[rel]).to('cuda')
subgraph

train_mask = subgraph.nodes['tweet'].data['train_mask']
val_mask = subgraph.nodes['tweet'].data['val_mask']
test_mask = subgraph.nodes['tweet'].data['test_mask']

class SAGEClassifier(nn.Module):
    def __init__(self, hidden_dim: int = 500):
        super().__init__()
        feats1 = subgraph.nodes[rel[0]].data['feat'].shape[-1]
        feats2 = subgraph.nodes[rel[2]].data['feat'].shape[-1]
        self.conv = dglnn.SAGEConv(in_feats=(feats1, feats2), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU())
        self.clf = nn.Sequential(
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )


    def forward(self, graph, x):
        x = self.conv(graph, (x['p1'], x['p2']))
        x = self.clf(x)
        return x

gnn = SAGEClassifier().cuda()
gnn

def forward_pass() -> dict:
    '''A forward pass of the graph neural network.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Set the GNN to training mode
    gnn.train()

    # Get the input features and the output labels
    input_feats = dict(
        p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
        p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
    )
    output_labels = subgraph.nodes['tweet'].data['label'].cuda()

    # Forward propagation
    logits = gnn(subgraph, input_feats).squeeze()

    # Compute loss
    loss = F.binary_cross_entropy_with_logits(
        input=logits[train_mask],
        target=output_labels.float()[train_mask]
    )

    # Compute training metrics
    scores = scorer(logits[train_mask].ge(0), output_labels[train_mask])
    misinformation_f1 = scores[0]
    factual_f1 = scores[1]

    return dict(loss=loss, 
                misinformation_f1=misinformation_f1, 
                factual_f1=factual_f1)

def evaluate(split: str) -> dict:
    '''Evaluate the graph neural network.

    Args:
        split (str):
            The split to evaluate the GNN on. Can be 'val' or 'test'.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Get the correct mask, depending on the value of `split`
    mask = val_mask if split == 'val' else test_mask

    gnn.eval()
    with torch.no_grad():

        # Get the input features and the output labels
        input_feats = dict(
            p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
            p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
        )
        output_labels = subgraph.nodes['tweet'].data['label'].cuda()

        # Forward propagation
        logits = gnn(subgraph, input_feats).squeeze()

        # Compute validation loss
        val_loss = F.binary_cross_entropy_with_logits(
            input=logits[mask],
            target=output_labels.float()[mask]
        ).cpu().item()

        # Compute validation metrics
        scores = scorer(logits[mask].ge(0), output_labels[mask])
        val_misinformation_f1 = scores[0].cpu().item()
        val_factual_f1 = scores[1].cpu().item()

    return dict(loss=val_loss, 
                misinformation_f1=val_misinformation_f1, 
                factual_f1=val_factual_f1)

# Initialise optimiser
opt = optim.AdamW(gnn.parameters(), lr=3e-4)

# Initialise scorer
scorer = tm.classification.f_beta.F1Score(num_classes=2, average='none').cuda()

# Initialise dictionary containing validation scores
val_scores = defaultdict(list)

# Initialise progress bar
epoch_pbar = tqdm(range(1000), desc='Training')

for epoch in epoch_pbar:

    # Reset the gradients
    opt.zero_grad()

    # Forward propagation
    train_results = forward_pass()

    # Backward propagation
    train_results['loss'].backward()

    # Update gradients
    opt.step()

    # Evaluate the model
    val_results = evaluate('val')

    # Store the validation scores
    for metric in ['loss', 'misinformation_f1', 'factual_f1']:
        val_scores[metric].append(val_results[metric])

    # Update progress bar description
    if epoch % 25 == 0 and epoch > 0:
        val_loss = np.mean(val_scores['loss'])
        val_misinformation_f1 = np.mean(val_scores['misinformation_f1'])
        val_factual_f1 = np.mean(val_scores['factual_f1'])
        desc = (f'Training - '
                f'loss {train_results["loss"]:.3f} - '
                f'factual_f1 {train_results["factual_f1"]:.3f} - '
                f'misinfo_f1 {train_results["misinformation_f1"]:.3f} - '
                f'val_loss {val_loss:.3f} - '
                f'val_factual_f1 {val_factual_f1:.3f} - '
                f'val_misinfo_f1 {val_misinformation_f1:.3f}')
        epoch_pbar.set_description(desc)
        val_scores = defaultdict(list)

test_results = evaluate('test')
macro_f1 = np.mean([test_results['factual_f1'],
                    test_results['misinformation_f1']])
print('*** Test results ***')
print(f'Factual F1: {test_results["factual_f1"]:.4f}')
print(f'Misinformation F1: {test_results["misinformation_f1"]:.4f}')
print(f'Macro-average F1: {macro_f1:.4f}')

Training:   0%|          | 0/1000 [00:00<?, ?it/s]

*** Test results ***
Factual F1: 0.1653
Misinformation F1: 0.8736
Macro-average F1: 0.5194


## 'user', 'mentions_inv', 'tweet'

In [ ]:
rel = ('user', 'mentions_inv', 'tweet')
subgraph = dgl.edge_type_subgraph(dgl_graph, etypes=[rel]).to('cuda')
subgraph

train_mask = subgraph.nodes['tweet'].data['train_mask']
val_mask = subgraph.nodes['tweet'].data['val_mask']
test_mask = subgraph.nodes['tweet'].data['test_mask']

class SAGEClassifier(nn.Module):
    def __init__(self, hidden_dim: int = 500):
        super().__init__()
        feats1 = subgraph.nodes[rel[0]].data['feat'].shape[-1]
        feats2 = subgraph.nodes[rel[2]].data['feat'].shape[-1]
        self.conv = dglnn.SAGEConv(in_feats=(feats1, feats2), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU())
        self.clf = nn.Sequential(
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )


    def forward(self, graph, x):
        x = self.conv(graph, (x['p1'], x['p2']))
        x = self.clf(x)
        return x

gnn = SAGEClassifier().cuda()
gnn

def forward_pass() -> dict:
    '''A forward pass of the graph neural network.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Set the GNN to training mode
    gnn.train()

    # Get the input features and the output labels
    input_feats = dict(
        p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
        p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
    )
    output_labels = subgraph.nodes['tweet'].data['label'].cuda()

    # Forward propagation
    logits = gnn(subgraph, input_feats).squeeze()

    # Compute loss
    loss = F.binary_cross_entropy_with_logits(
        input=logits[train_mask],
        target=output_labels.float()[train_mask]
    )

    # Compute training metrics
    scores = scorer(logits[train_mask].ge(0), output_labels[train_mask])
    misinformation_f1 = scores[0]
    factual_f1 = scores[1]

    return dict(loss=loss, 
                misinformation_f1=misinformation_f1, 
                factual_f1=factual_f1)

def evaluate(split: str) -> dict:
    '''Evaluate the graph neural network.

    Args:
        split (str):
            The split to evaluate the GNN on. Can be 'val' or 'test'.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Get the correct mask, depending on the value of `split`
    mask = val_mask if split == 'val' else test_mask

    gnn.eval()
    with torch.no_grad():

        # Get the input features and the output labels
        input_feats = dict(
            p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
            p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
        )
        output_labels = subgraph.nodes['tweet'].data['label'].cuda()

        # Forward propagation
        logits = gnn(subgraph, input_feats).squeeze()

        # Compute validation loss
        val_loss = F.binary_cross_entropy_with_logits(
            input=logits[mask],
            target=output_labels.float()[mask]
        ).cpu().item()

        # Compute validation metrics
        scores = scorer(logits[mask].ge(0), output_labels[mask])
        val_misinformation_f1 = scores[0].cpu().item()
        val_factual_f1 = scores[1].cpu().item()

    return dict(loss=val_loss, 
                misinformation_f1=val_misinformation_f1, 
                factual_f1=val_factual_f1)

# Initialise optimiser
opt = optim.AdamW(gnn.parameters(), lr=3e-4)

# Initialise scorer
scorer = tm.classification.f_beta.F1Score(num_classes=2, average='none').cuda()

# Initialise dictionary containing validation scores
val_scores = defaultdict(list)

# Initialise progress bar
epoch_pbar = tqdm(range(1000), desc='Training')

for epoch in epoch_pbar:

    # Reset the gradients
    opt.zero_grad()

    # Forward propagation
    train_results = forward_pass()

    # Backward propagation
    train_results['loss'].backward()

    # Update gradients
    opt.step()

    # Evaluate the model
    val_results = evaluate('val')

    # Store the validation scores
    for metric in ['loss', 'misinformation_f1', 'factual_f1']:
        val_scores[metric].append(val_results[metric])

    # Update progress bar description
    if epoch % 25 == 0 and epoch > 0:
        val_loss = np.mean(val_scores['loss'])
        val_misinformation_f1 = np.mean(val_scores['misinformation_f1'])
        val_factual_f1 = np.mean(val_scores['factual_f1'])
        desc = (f'Training - '
                f'loss {train_results["loss"]:.3f} - '
                f'factual_f1 {train_results["factual_f1"]:.3f} - '
                f'misinfo_f1 {train_results["misinformation_f1"]:.3f} - '
                f'val_loss {val_loss:.3f} - '
                f'val_factual_f1 {val_factual_f1:.3f} - '
                f'val_misinfo_f1 {val_misinformation_f1:.3f}')
        epoch_pbar.set_description(desc)
        val_scores = defaultdict(list)

test_results = evaluate('test')
macro_f1 = np.mean([test_results['factual_f1'],
                    test_results['misinformation_f1']])
print('*** Test results ***')
print(f'Factual F1: {test_results["factual_f1"]:.4f}')
print(f'Misinformation F1: {test_results["misinformation_f1"]:.4f}')
print(f'Macro-average F1: {macro_f1:.4f}')

Training:   0%|          | 0/1000 [00:00<?, ?it/s]

*** Test results ***
Factual F1: 0.0000
Misinformation F1: 0.9486
Macro-average F1: 0.4743


## 'user', 'retweeted', 'tweet'

In [ ]:
rel = ('user', 'retweeted', 'tweet')
subgraph = dgl.edge_type_subgraph(dgl_graph, etypes=[rel]).to('cuda')
subgraph

train_mask = subgraph.nodes['tweet'].data['train_mask']
val_mask = subgraph.nodes['tweet'].data['val_mask']
test_mask = subgraph.nodes['tweet'].data['test_mask']

class SAGEClassifier(nn.Module):
    def __init__(self, hidden_dim: int = 500):
        super().__init__()
        feats1 = subgraph.nodes[rel[0]].data['feat'].shape[-1]
        feats2 = subgraph.nodes[rel[2]].data['feat'].shape[-1]
        self.conv = dglnn.SAGEConv(in_feats=(feats1, feats2), 
                                   out_feats=hidden_dim, 
                                   aggregator_type='lstm',
                                   activation=nn.GELU())
        self.clf = nn.Sequential(
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, 1)
        )


    def forward(self, graph, x):
        x = self.conv(graph, (x['p1'], x['p2']))
        x = self.clf(x)
        return x

gnn = SAGEClassifier().cuda()
gnn

def forward_pass() -> dict:
    '''A forward pass of the graph neural network.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Set the GNN to training mode
    gnn.train()

    # Get the input features and the output labels
    input_feats = dict(
        p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
        p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
    )
    output_labels = subgraph.nodes['tweet'].data['label'].cuda()

    # Forward propagation
    logits = gnn(subgraph, input_feats).squeeze()

    # Compute loss
    loss = F.binary_cross_entropy_with_logits(
        input=logits[train_mask],
        target=output_labels.float()[train_mask]
    )

    # Compute training metrics
    scores = scorer(logits[train_mask].ge(0), output_labels[train_mask])
    misinformation_f1 = scores[0]
    factual_f1 = scores[1]

    return dict(loss=loss, 
                misinformation_f1=misinformation_f1, 
                factual_f1=factual_f1)

def evaluate(split: str) -> dict:
    '''Evaluate the graph neural network.

    Args:
        split (str):
            The split to evaluate the GNN on. Can be 'val' or 'test'.

    Returns:
        dict:
            A dict with keys 'loss', 'misinformation_f1' and 'factual_f1', 
            with values as their corresponding values.
    '''
    # Get the correct mask, depending on the value of `split`
    mask = val_mask if split == 'val' else test_mask

    gnn.eval()
    with torch.no_grad():

        # Get the input features and the output labels
        input_feats = dict(
            p1=subgraph.nodes[rel[0]].data['feat'].float().cuda(),
            p2=subgraph.nodes[rel[2]].data['feat'].float().cuda()
        )
        output_labels = subgraph.nodes['tweet'].data['label'].cuda()

        # Forward propagation
        logits = gnn(subgraph, input_feats).squeeze()

        # Compute validation loss
        val_loss = F.binary_cross_entropy_with_logits(
            input=logits[mask],
            target=output_labels.float()[mask]
        ).cpu().item()

        # Compute validation metrics
        scores = scorer(logits[mask].ge(0), output_labels[mask])
        val_misinformation_f1 = scores[0].cpu().item()
        val_factual_f1 = scores[1].cpu().item()

    return dict(loss=val_loss, 
                misinformation_f1=val_misinformation_f1, 
                factual_f1=val_factual_f1)

# Initialise optimiser
opt = optim.AdamW(gnn.parameters(), lr=3e-4)

# Initialise scorer
scorer = tm.classification.f_beta.F1Score(num_classes=2, average='none').cuda()

# Initialise dictionary containing validation scores
val_scores = defaultdict(list)

# Initialise progress bar
epoch_pbar = tqdm(range(1000), desc='Training')

for epoch in epoch_pbar:

    # Reset the gradients
    opt.zero_grad()

    # Forward propagation
    train_results = forward_pass()

    # Backward propagation
    train_results['loss'].backward()

    # Update gradients
    opt.step()

    # Evaluate the model
    val_results = evaluate('val')

    # Store the validation scores
    for metric in ['loss', 'misinformation_f1', 'factual_f1']:
        val_scores[metric].append(val_results[metric])

    # Update progress bar description
    if epoch % 25 == 0 and epoch > 0:
        val_loss = np.mean(val_scores['loss'])
        val_misinformation_f1 = np.mean(val_scores['misinformation_f1'])
        val_factual_f1 = np.mean(val_scores['factual_f1'])
        desc = (f'Training - '
                f'loss {train_results["loss"]:.3f} - '
                f'factual_f1 {train_results["factual_f1"]:.3f} - '
                f'misinfo_f1 {train_results["misinformation_f1"]:.3f} - '
                f'val_loss {val_loss:.3f} - '
                f'val_factual_f1 {val_factual_f1:.3f} - '
                f'val_misinfo_f1 {val_misinformation_f1:.3f}')
        epoch_pbar.set_description(desc)
        val_scores = defaultdict(list)

test_results = evaluate('test')
macro_f1 = np.mean([test_results['factual_f1'],
                    test_results['misinformation_f1']])
print('*** Test results ***')
print(f'Factual F1: {test_results["factual_f1"]:.4f}')
print(f'Misinformation F1: {test_results["misinformation_f1"]:.4f}')
print(f'Macro-average F1: {macro_f1:.4f}')

Training:   0%|          | 0/1000 [00:00<?, ?it/s]

*** Test results ***
Factual F1: 0.1786
Misinformation F1: 0.9468
Macro-average F1: 0.5627


## performance

In [ ]:
model_performance = dict()
model_performance['user_posted_tweet'] = dict(zip(('Factual F1', 'Misinformation F1', 'Macro-average F1', 'Training Time'), (0.1633, 0.9529, 0.5581, 53)))
model_performance['reply_reply_to_tweet'] = dict(zip(('Factual F1', 'Misinformation F1', 'Macro-average F1', 'Training Time'), (0.1644, 0.9280, 0.5462, 6923)))
model_performance['claim_discusses_inv_tweet'] = dict(zip(('Factual F1', 'Misinformation F1', 'Macro-average F1', 'Training Time'), (0.1044, 0.4137, 0.2590, 92)))
model_performance['reply_quote_of_tweet'] = dict(zip(('Factual F1', 'Misinformation F1', 'Macro-average F1', 'Training Time'), (0.1791, 0.9355, 0.5573, 7804)))
model_performance['article_has_article_inv_tweet'] = dict(zip(('Factual F1', 'Misinformation F1', 'Macro-average F1', 'Training Time'), (0.2703, 0.9694, 0.6198, 78)))
model_performance['hashtag_has_hashtag_inv_tweet'] = dict(zip(('Factual F1', 'Misinformation F1', 'Macro-average F1', 'Training Time'), (0.1250, 0.9685, 0.5467, 38)))
model_performance['image_has_image_inv_tweet'] = dict(zip(('Factual F1', 'Misinformation F1', 'Macro-average F1', 'Training Time'), (0.1653, 0.8736, 0.5194, 37)))
model_performance['user_mentions_inv_tweet'] = dict(zip(('Factual F1', 'Misinformation F1', 'Macro-average F1', 'Training Time'), (0.0000, 0.9486, 0.4743, 75)))
model_performance['user_retweeted_tweet'] = dict(zip(('Factual F1', 'Misinformation F1', 'Macro-average F1', 'Training Time'), (0.1786, 0.9468, 0.5627, 211)))


In [ ]:
import pandas as pd
performance = pd.DataFrame(model_performance).T
performance.sort_values(by=['Macro-average F1'], ascending=False)

,Factual F1,Misinformation F1,Macro-average F1,Training Time
article_has_article_inv_tweet,0.2703,0.9694,0.6198,78.0
user_retweeted_tweet,0.1786,0.9468,0.5627,211.0
user_posted_tweet,0.1633,0.9529,0.5581,53.0
reply_quote_of_tweet,0.1791,0.9355,0.5573,7804.0
hashtag_has_hashtag_inv_tweet,0.1250,0.9685,0.5467,38.0
reply_reply_to_tweet,0.1644,0.9280,0.5462,6923.0
image_has_image_inv_tweet,0.1653,0.8736,0.5194,37.0
user_mentions_inv_tweet,0.0000,0.9486,0.4743,75.0
claim_discusses_inv_tweet,0.1044,0.4137,0.2590,92.0
